In [3]:
from learning_agent import robo_scientist
from theories import theory_feynman
from theories import theory_polynomial1D
from theories import theory_polynomial2D
from theories import theory_nested_formulas
from theories import theory_master
from environments import ohm_law, single_param_linear_law, env_1, sin, tg, arcsin, cos, env_2
from data_generator import simple_generator, std_generator
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(context="paper", style="whitegrid", font_scale=2)


In [4]:
feynman_dir = os.path.join(os.getcwd(), 'theories/feynman/')
working_dirs = {
    theory_feynman.TheoryFeynman: feynman_dir,
}

In [5]:
class Analyser(object):
    def stats_for_env(self, filename, env_cls, theories_with_params,
                      env_params, iterations_count, labels=None, maketitle=True):
        feynman_dir = os.path.join(os.getcwd(), 'theories/feynman/')
        working_dirs = {
            theory_feynman.TheoryFeynman: feynman_dir,
        }
        rs = robo_scientist.RoboScientist(working_dirs, keep_full_history=True)
        mses = {}
        for t_with_params in theories_with_params:
            t, t_params = t_with_params[0], t_with_params[1]
            print(t)
            include_derivatives = (t == theory_polynomial1D.TheoryPolynomial1D
                                   or
                                   t == theory_polynomial2D.TheoryPolynomial2D)
            env = env_cls(**env_params,
                          include_derivatives=include_derivatives)
            gen = simple_generator.SimpleGenerator
            if t == theory_master.MasterTheory:
                gen = std_generator.STDGenerator
            rs.explore_environment(env, t, gen,
                                   iterations_count, t_params)
            h = rs.get_history_for_exploration_key(
                robo_scientist.ExplorationKey(env_cls.__name__, t.__name__))
            mses[(t.__name__, str(t_params))] = np.array([a.mse for a in h])
            print(t.__name__, str(t_params))

        colors = ['r', 'g', 'b', 'purple', 'black', 'tan', 'turquoise']
        f = plt.figure(figsize=(18, 8), dpi=100)
        inf = 80
        for i in range(len(mses)):
            m = mses[(theories_with_params[i][0].__name__,
                      str(theories_with_params[i][1]))]
            log_mses = np.where(m > 0, np.log(m), -inf)
            if labels is not None:
                plt.plot(log_mses,
                         c=colors[i],
                         label=labels[i])
            else:
                plt.plot(log_mses,
                         c=colors[i],
                         label='{}, params: {}'.format(
                             theories_with_params[i][0].__name__,
                             theories_with_params[i][1]))
        if maketitle:
            plt.title('Environment: {}, params: {}'.format(env_cls.__name__,
                                                           env_params))
        plt.xlabel('iterations count (dataset size)')
        plt.ylabel('log MSE')
        plt.legend()
        #         plt.tick_params(labelsize=15)
        plt.tight_layout()
        f.savefig(filename, dpi=f.dpi)
        plt.show()

In [6]:
a = Analyser()

In [7]:
# theories = [
#     (theory_feynman.TheoryFeynman, {}), 
# #     (theory_nested_formulas.TheoryNestedFormula, {}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Fourier', 'polynomial_cnt': 5}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 2}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 3}),
#     (theory_polynomial1D.TheoryPolynomial1D, 
#      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 4}),
#     (theory_polynomial1D.TheoryPolynomial1D, 
#      {'polynomial_type': 'Fourier', 'polynomial_cnt': 7}),
#     (theory_polynomial1D.TheoryPolynomial1D, 
#      {'polynomial_type': 'Fourier', 'polynomial_cnt': 12}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 6}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 8}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 10}),
# #     (theory_polynomial1D.TheoryPolynomial1D, 
# #      {'polynomial_type': 'Chebyshev', 'polynomial_cnt': 20}),
# ]

# a.stats_for_env('f1', ohm_law.OhmLawEnvironment, theories, {'resistance': 0.57}, 100)

In [8]:
p_values = [10, 20]
master_theories = [
    (
        theory_master.MasterTheory, 
        {"theories_with_params": [
            (theory_nested_formulas.TheoryNestedFormula, None) for i in range(p)
        ]}
    ) for p in p_values
]

theories = [
    (theory_feynman.TheoryFeynman, {'BF_try_time': 5}),
    (theory_nested_formulas.TheoryNestedFormula, {})
] + master_theories

labels = [
    "AI Feynman", "nested formula of depth 1, random design"
] + [
    "Master nested formulas of depth {}, design based on maximal variance".format(p) for p in p_values
]

In [ ]:
available_environments = [
    ohm_law.OhmLawEnvironment, single_param_linear_law.LinearLawEnvironment, 
    env_1.Environment1, sin.SinEnvironment, tg.TgEnvironment, 
    arcsin.ArcsinEnvironment, cos.CosEnvironment, env_2.Environment2
]
params = [
    {"resistance": 0.57}, {"a": 12, "b": 44}, 
    {}, {}, {},
    {}, {}, {},
]
filenames = [ 
    "env1", "sin", "tg",
    "arcsin", "cos", "env2"
]
for i, available_environment in enumerate(available_environments):
    a.stats_for_env(filenames[i], available_environment, theories, params[i], 
                    100, labels=labels, maketitle=False)    

<class 'theories.theory_feynman.TheoryFeynman'>
TheoryFeynman {'BF_try_time': 5}
<class 'theories.theory_nested_formulas.TheoryNestedFormula'>
TheoryNestedFormula {}
<class 'theories.theory_master.MasterTheory'>


/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


MasterTheory {'theories_with_params': [(<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None), (<class 'theories.theory_nested_formulas.TheoryNestedFormula'>, None)]}
<class 'theories.theory_master.MasterTheory'>


/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:256: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred)